### 1) Use Selenium and BeautifulSoup to read the contents of the HTML.

In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import random
from time import sleep

from bs4 import BeautifulSoup
import urllib

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.headless = True

import os

chromedriver = "./chromedriver/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

### 2) Obtain the hyperlinks for 4,000 Data related job roles

In [140]:
# First, create an empty DataFrame with following columns.
jobs = pd.DataFrame(columns=['link', 'employer', 'location', 'employment_type', 'seniority', 'sector', 
                             'pay_min', 'pay_max', 'pay_type'])

In [146]:
for i in range (50,200):
    
    if i%20==0:
        print (i)
        
    if i!=0 and i%50==0:
            jobs.to_csv("./jobs{}.csv" .format(i))
    
    # Set the URL we want to visit.
    url = "https://www.mycareersfuture.sg/search?search=data&sortBy=new_posting_date&page="+str(i)
    
    # Visit page with Selenium driver.
    driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver", chrome_options=options)
    driver.get(url)
    
    # Wait 15 seconds.
    sleep(15)

    # Grab the page source.
    html = driver.page_source
    
    # Pass page source to Beautifulsoup
    listing = BeautifulSoup(html, 'html.parser')
    
    # Close Driver
    driver.close()

    # The data we are looking for are all located in this tag and class
    main = listing.find('div', {'class':'card-list'})
    
    # Loop through each entry.
    for entry in main.find_all('div', id=True):

        # Link
        try:
            link = entry.find('a', {'class':'bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3'})['href']
        except:
            link = np.nan
        
        # Employer
        try:
            employer = entry.find('p', {'name':'company'}).text
        except:
            employer = np.nan
        
        # Location
        try:
            location = entry.find('p', {'name':'location'}).text
        except:
            location = np.nan

        # Employment Type
        try:
            employment_type = entry.find('p', {'name':'employment_type'}).text
        except:
            employment_type = np.nan

        # Seniority
        try:
            seniority = entry.find('p', {'name':'seniority'}).text
        except:
            seniority = np.nan
            
        # Sector
        try:
            sector = entry.find('p', {'name':'category'}).text
        except:
            sector = np.nan
            
        # Min Pay
        try:
            pay_min = entry.find('div', {'class':'lh-solid'}).text.split('to')[0].replace('$','').replace(',','')
        except:
            pay_min = np.nan
            
        # Max Pay
        try:
            pay_max = entry.find('div', {'class':'lh-solid'}).text.split('to')[1].replace('$','').replace(',','')
        except:
            pay_max = np.nan
            
        # Pay Type
        try:
            pay_type = entry.find('span', {'class':'salary_type black-60 mv0 fw6 f5-5 lh-solid db pb2 dib i'}).text
        except:
            pay_type = np.nan
            
        # Update Dataframe with listing information
        jobs.loc[len(jobs)] = [link, employer, location, employment_type, seniority, sector, pay_min, 
                               pay_max, pay_type]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


60
80
100
120
140
160
180


In [16]:
jobs = pd.read_csv("./jobs.csv")

jobs.drop(['Unnamed: 0'], axis=1, inplace=True)

In [17]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
link               4000 non-null object
employer           4000 non-null object
location           3856 non-null object
employment_type    4000 non-null object
seniority          3932 non-null object
sector             4000 non-null object
pay_min            3567 non-null float64
pay_max            3567 non-null float64
pay_type           3567 non-null object
dtypes: float64(2), object(7)
memory usage: 281.3+ KB


In [18]:
jobs.head()

,link,employer,location,employment_type,seniority,sector,pay_min,pay_max,pay_type
0,/job/data-scientist-singapore-power-5731c18be1...,SINGAPORE POWER LIMITED,Central,Full Time,Professional,Information Technology,4000.0,8000.0,Monthly
1,/job/senior-data-scientist-singapore-power-b8f...,SINGAPORE POWER LIMITED,Central,Full Time,Middle Management,Information Technology,8000.0,14000.0,Monthly
2,/job/data-director-tbwa-singapore-14652381c7eb...,TBWA SINGAPORE PTE LTD,South,Permanent ...,Manager ...,Advertising / Media,8000.0,15000.0,Monthly
3,/job/head-data-engineering-fixed-mobile-09c037...,FIXED & MOBILE PTE. LTD.,Central,Permanent ...,Senior Management,Information Technology,8000.0,11000.0,Monthly
4,/job/data-modeler-monarch-info-tech-services-9...,MONARCH INFO TECH SERVICES PTE. LTD.,Central,Full Time,Professional,Information Technology,6000.0,6500.0,Monthly


### 3) Obtain job deatils for each of the 4,000 Data related job roles scrapped in Step 2

In [64]:
# Create additional columns in jobs Dataframe
jobs = jobs.reindex(columns=['link', 'employer', 'location', 'employment_type', 'seniority', 'sector', 
                             'pay_min', 'pay_max', 'pay_type', 'about', 'rr', 'requirements'])

In [7]:
for i in range (1860,len(jobs)):
    
    if i%50==0:
        print (i)
        
    if i!=0 and i%300==0:
            jobs.to_csv("./jobs{}.csv" .format(i))
    
    # Set the URL of the listing.
    url = "https://www.mycareersfuture.sg"+jobs.loc[i]['link']

    # Visit page with Selenium driver.
    driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver", options=options)
    driver.get(url)

    # Wait 8 seconds.
    sleep(8)

    # Grab the page source.
    html = driver.page_source

    # Pass page source to Beautifulsoup
    listing = BeautifulSoup(html, 'html.parser')

    # Close Driver
    driver.close()
    
    #Extract information from listing

    # About Company
    try:
        about = listing.find('div', {'class':'black-70 f6 lh-copy break-word op-after pt1 mh100-sp of-hide-sp'}).find_all('p')[1].text
    except:
        about = np.nan
   
    # Roles & Responsibilities
    try:
        rr = listing.find('div', {'id':'description-content'}).text
    except:
        rr = np.nan
    
    # Requirements
    try:
        requirements = listing.find('div', {'id':'requirements-content'}).text
    except:
        requirements = np.nan
        
    # Update Dataframe with listing information
    jobs.loc[i,['about', 'rr', 'requirements']] = [about, rr, requirements]

1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950


In [9]:
jobs.to_csv("./jobslist.csv", index=False)